In [ ]:
import np
import vep_utils

"""
Pattern reversal Load and Visualize Data
===============================

This example demonstrates loading, organizing, and visualizing EP response data from the visual P100 experiment. 

An animation of a checkerboard reversal is shown(the checkerboard squares' colours are toggled once each half a second).

The data used is the first subject and first session of the one of the eeg-notebooks P100 example datasets.
It was recorded using an OpenBCI Ultracortex EEG headset(Mark IV) with it's last five electrodes placed in the headset's
node locations of (PO1, Oz, PO2, P3 and P4).
These headset node locations were used to fit around a Meta Quest 2 headset, which tilted/angled the headset backwards
so that the real locations of the electrodes are closer to the occipital lobe - O1, Iz, O2, PO1 and PO2.
The session consisted of using the Meta Quest 2 linked with a PC to display the checkerboard reversal animation
for thirty seconds of continuous recording.  

We first use the `fetch_datasets` to obtain a list of filenames. If these files are not already present 
in the specified data directory, they will be quickly downloaded from the cloud. 

After loading the data from the occiptal channels, we place it in an MNE `Epochs` object, and then an `Evoked` object to obtain
the trial-averaged delay of the response. 

The final figure plotted at the end shows the P100 response EP waveform.
"""

###################################################################################################
# Setup
# ---------------------

# Some standard pythonic imports
import os
from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

# MNE functions
from mne import Epochs,find_events

# EEG-Notebooks functions
from eegnb.analysis.utils import load_data
from vep_utils import plot_vep
from os import path, getenv

###################################################################################################
# Load Data
# ---------------------
#
# We will use the eeg-notebooks P100 example dataset
#
# Note that if you are running this locally, the following cell will download
# the example dataset, if you do not already have it.
#

###################################################################################################

data_dir = path.join(path.expanduser("~/"), getenv('DATA_DIR'), "data")
raw = load_data(subject=0,session=1,
                experiment='block_both_eyes_pattern_reversal-mark_iv_headset', site='windows_acer_34_100hz', device_name='cyton',
                data_dir=data_dir)

In [ ]:
###################################################################################################
# Visualize the power spectrum
# ----------------------------

raw.plot_psd()

###################################################################################################
# Filtering
# ----------------------------

raw.filter(1,30, method='fir')
raw.plot_psd(fmin=1, fmax=30)

In [ ]:

###################################################################################################
# Epoching
# ----------------------------

# Create an array containing the timestamps and which eye was presented the stimulus
events = find_events(raw)
event_id = {'left_eye': 1, 'right_eye': 2}

# Create an MNE Epochs object representing all the epochs around stimulus presentation
epochs = Epochs(raw, events=events, event_id=event_id,
                tmin=-0.1, tmax=0.4, baseline=None,
                reject={'eeg': 65e-6}, preload=True,
                verbose=False, picks=[7])
epochs.shift_time(-vep_utils.windows_lag())
print('sample drop %: ', (1 - len(epochs.events)/len(events)) * 100)
epochs

In [ ]:
###################################################################################################
# Epoch average
# ----------------------------
evoked = epochs.average()
evoked.plot(spatial_colors=True, show=False)

In [ ]:
evoked_potentials = epochs['left_eye'].average(picks=['Oz'])
plot_vep(evoked_potentials)

In [ ]:
evoked_potentials = epochs['right_eye'].average(picks=['Oz'])
plot_vep(evoked_potentials)

In [ ]:
###################################################################################################
# Compare evoked potentials by event type
# ----------------------------

# Create separate evoked responses for each event type
evoked_left = epochs['left_eye'].average(picks=['Oz'])
evoked_right = epochs['right_eye'].average(picks=['Oz'])

# Plot both conditions on the same figure for comparison
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))

# Extract time points and data
times = evoked_left.times * 1000  # Convert to milliseconds
left_data = evoked_left.data[0] * 1e6  # Convert to microvolts
right_data = evoked_right.data[0] * 1e6  # Convert to microvolts

# Plot both conditions
ax.plot(times, left_data, label='Left Eye', color='blue', linewidth=2)
ax.plot(times, right_data, label='Right Eye', color='red', linewidth=2)

# Add formatting
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Amplitude (μV)')
ax.set_title('Comparison of Evoked Potentials: Left Eye vs Right Eye')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax.axvline(x=0, color='black', linestyle='--', alpha=0.5, label='Stimulus Onset')

plt.tight_layout()
plt.show()

# Print summary statistics
print(f"Left eye - Number of epochs: {len(epochs['left_eye'])}")
print(f"Right eye - Number of epochs: {len(epochs['right_eye'])}")

# Find P100 peak for each condition (typically around 100ms)
p100_window = (80, 120)  # milliseconds
time_mask = (times >= p100_window[0]) & (times <= p100_window[1])

left_p100_idx = np.argmax(left_data[time_mask])
right_p100_idx = np.argmax(right_data[time_mask])

left_p100_time = times[time_mask][left_p100_idx]
left_p100_amp = left_data[time_mask][left_p100_idx]

right_p100_time = times[time_mask][right_p100_idx]
right_p100_amp = right_data[time_mask][right_p100_idx]

print(f"\nP100 Peak Analysis:")
print(f"Left eye - Peak at {left_p100_time:.1f}ms, amplitude: {left_p100_amp:.2f}μV")
print(f"Right eye - Peak at {right_p100_time:.1f}ms, amplitude: {right_p100_amp:.2f}μV")

In [ ]:
###################################################################################################
# Create difference wave
# ----------------------------

# Calculate the difference between conditions
difference_data = left_data - right_data

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, difference_data, label='Left - Right', color='green', linewidth=2)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Amplitude Difference (μV)')
ax.set_title('Difference Wave: Left Eye - Right Eye')
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax.axvline(x=0, color='black', linestyle='--', alpha=0.5, label='Stimulus Onset')
ax.legend()

plt.tight_layout()
plt.show()